In [1]:
import sys, os
import numpy as np
import glob
import pandas as pd
import math

from scipy import stats
import matplotlib.pyplot as plt
from astropy.io import fits

from bokeh.io import output_notebook, show, export_png,export_svgs, save
from bokeh.plotting import figure, show, output_file
import bokeh.palettes
from bokeh.layouts import column,gridplot
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.models.glyphs import Text
from bokeh.palettes import Category20
from bokeh.transform import linear_cmap

from ics.cobraCharmer import pfi as pfiControl
from astropy.table import Table


In [2]:
class ontimeModel():
    
    def getPhiFwdSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid]

        onTimeArray = ndf['J2onTime'].values
        angSpdArray = ndf['J2_fwd'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def getPhiRevSlope(self, pid, dataframe):

        ndf = dataframe.loc[dataframe['fiberNo'] == pid]

        onTimeArray = ndf['J2onTime'].values
        angSpdArray = ndf['J2_rev'].values

        slope, intercept, r_value, p_value, std_err = stats.linregress(onTimeArray,angSpdArray)

        # If the slope is nan, that means the linear regression is failed.  Return zero instead of nan.
        if math.isnan(slope) is True:
            slope = 0

        return slope,intercept

    def buildModelfromDataFrame(self, dataframe, visibleFibers=False):

        # Reading all model and build a model list

        j2fwd_slope = []
        j2fwd_int = []
        
        j2rev_slope = []
        j2rev_int = []


        for pid in range(1,58):
            fw_s, fw_i=self.getPhiFwdSlope(pid,dataframe)
            rv_s, rv_i=self.getPhiRevSlope(pid,dataframe)
            
            j2fwd_slope.append(fw_s)
            j2fwd_int.append(fw_i)
            
            j2rev_slope.append(rv_s)
            j2rev_int.append(rv_i)
       
        self.j2fwd_slope = j2fwd_slope
        self.j2fwd_int   = j2fwd_int
        self.j2rev_slope = j2rev_slope
        self.j2rev_int   = j2rev_int


    def getTargetOnTime(self, target, modelSlope, onTime, angSpeed):

        
        size = len(onTime)
        newOntime_ms = np.zeros(size)
        sumx=np.zeros(size)

        onTime_ms = onTime*1000
        #if (target.any() > 0) :
        if isinstance(modelSlope, list) ==  True:
 
            for i in range(size):
                if modelSlope[i] == 0:
                    sumx[i]=0
                else:
                    sumx[i] = (target - angSpeed[i])/modelSlope[i]   

        else:
            sumx = (target - angSpeed)/modelSlope  

        newOntime_ms= onTime_ms+sumx

        newOntime = newOntime_ms / 1000.0 
        
        
        return newOntime


In [3]:
def plotJ2OntimeSpeed(GroupIdx, dataFrame, xrange, yrange):

    mapper = Category20[len(GroupIdx)]
    TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

    p = figure( tools=TOOLS, x_range=xrange, y_range=yrange,plot_height=500, plot_width=1000)

    colorcode = 0
    for i in GroupIdx:
        legendname="Fiber "+str(i)

        p.line(x=dataFrame['J2onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J2_fwd'][dataFrame['fiberNo'] == i],\
        color=mapper[colorcode],line_width=2,legend=legendname)
        p.circle(x=dataFrame['J2onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J2_fwd'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)
        p.line(x=dataFrame['J2onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J2_rev'][dataFrame['fiberNo'] == i],color=mapper[colorcode],line_width=2)
        p.circle(x=dataFrame['J2onTime'][dataFrame['fiberNo'] == i], y=dataFrame['J2_rev'][dataFrame['fiberNo'] == i],radius=0.1,\
            color=mapper[colorcode],fill_color=None)

        colorcode = colorcode + 1

    return p


In [4]:
path='/Volumes/Disk/Data/20190530/phi/'

In [5]:
brokens = []
visibles= [e for e in range(1,58) if e not in brokens]
goodIdx = np.array(visibles) - 1
goodGroupIdx = {}
for group in range(6):
    goodGroupIdx[group] = goodIdx[goodIdx%6==group] + 1


In [6]:
dataarray=[]
pidarray=[]
otarray=[]
j2fwarray=[]
j2rvarray=[]
tarray=[15,25,35,45]

for tms in tarray:
    #print(tms)
    fw_file=path+f'{tms:0>4d}'+'/phiSpeedFW.npy'
    rv_file=path+f'{tms:0>4d}'+'/phiSpeedRv.npy'
    J2_fwd=np.load(fw_file)*180.0/math.pi 
    J2_rev=-np.load(rv_file)*180.0/math.pi 
    
    J2onTime=np.zeros(len(J2_rev))+tms
    pid=np.array(visibles)
    
    pidarray.append(pid)
    otarray.append(J2onTime)
    j2fwarray.append(J2_fwd)
    j2rvarray.append(J2_rev)
    
d={'fiberNo': np.array(pidarray).flatten(),'J2onTime':  np.array(otarray).flatten(), 
    'J2_fwd': np.array(j2fwarray).flatten(), 'J2_rev': np.array(j2rvarray).flatten()}
df = pd.DataFrame(d)

TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset','hover']

phirange = tarray

p1 = plotJ2OntimeSpeed(goodGroupIdx[0], df, [phirange[0], phirange[-1]], [-1,1])
p2 = plotJ2OntimeSpeed(goodGroupIdx[1], df, [phirange[0], phirange[-1]], [-1,1])
p3 = plotJ2OntimeSpeed(goodGroupIdx[2], df, [phirange[0], phirange[-1]], [-1,1])
p4 = plotJ2OntimeSpeed(goodGroupIdx[3], df, [phirange[0], phirange[-1]], [-1,1])
p5 = plotJ2OntimeSpeed(goodGroupIdx[4], df, [phirange[0], phirange[-1]], [-1,1])
p6 = plotJ2OntimeSpeed(goodGroupIdx[5], df, [phirange[0], phirange[-1]], [-1,1])

x = np.array([])
y1 = np.array([])
y2 = np.array([])
for tms in phirange:
    #print(np.mean(df['J2_fwd'][df['onTime']==tms].values))
    x=np.append(x,tms)
    y1=np.append(y1,np.median(df['J2_fwd'][df['J2onTime']==tms].values))
    y2=np.append(y2,np.median(df['J2_rev'][df['J2onTime']==tms].values))

slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y1[:8])
#fit=np.polyfit(x, y*100, 1)
#print(fit)

q = figure(tools=TOOLS, x_range=[phirange[0]-10, phirange[-1]+10], y_range=[-1.5,1.5],plot_height=500, plot_width=1000)
q.circle(x=df['J2onTime'], y=df['J2_fwd'],radius=0.3,\
        color='red',fill_color=None)
q.circle(x=x,y=y1, radius=0.5, color='blue')
legendtext=f'Y={slope:.8f}X{intercept:.2f}'
q.line(x=x[:8], y=x[:8]*slope+intercept,color='blue',line_width=3, legend=legendtext)

slope, intercept, r_value, p_value, std_err = stats.linregress(x[:8],y2[:8])
#fit=np.polyfit(x, y*100, 1)
q.circle(x=df['J2onTime'], y=df['J2_rev'],radius=0.3,\
        color='green',fill_color=None)
q.circle(x=x,y=y2, radius=0.5, color='#3B0F6F')
legendtext=f'Y={slope:.8f}X+{intercept:.2f}'
q.line(x=x[:8], y=x[:8]*slope+intercept,color='#3B0F6F',line_width=3, legend=legendtext)

output_file("phi.html")
save(column(p1,p2,p3,p4,p5,p6,q), filename="phi.html", \
    title='Phi On-time')


'/Users/chyan/Documents/workspace/ics_cobraCharmer/notebooks/phi.html'

In [7]:
otm = ontimeModel()
otm.buildModelfromDataFrame(df)

fwd_target = np.zeros(len(otm.j2fwd_int))+0.07
rev_target = np.zeros(len(otm.j2fwd_int))-0.07
newOntimeFwd2 = (fwd_target-otm.j2fwd_int)/otm.j2fwd_slope
newOntimeRev2 = (rev_target-otm.j2rev_int)/otm.j2rev_slope

newOntimeRev2[18]=60
#newOntimeFwd2 = otm.getTargetOnTime(0.07,otm.j2fwd_slope, model.motorOntimeFwd2 ,j2fwd_avg)
#newOntimeRev2 = otm.getTargetOnTime(-0.07,otm.j2rev_slope, model.motorOntimeRev2 ,j2rev_avg)


In [8]:
newOntimeRev2

array([28.51757556, 18.21713901, 19.59292609, 18.22767023, 16.82903661,
       21.14151778, 14.05586631, 24.67476551, 21.54133929, 21.27127363,
       20.0209118 , 25.48078282, 17.34635321, 22.04758428, 15.62708566,
       19.82216816, 23.85082416, 20.21590414, 60.        , 18.39672172,
       17.96722983, 16.244528  , 11.4728163 , 26.43745167, 15.04903235,
       27.28336027, 28.51757556, 17.66462965, 11.07520496, 22.09494086,
       14.16626763, 18.37331646, 20.71177299, 22.29653588, 23.55096343,
       26.29238431, 28.64614415, 26.06681988, 21.35602736, 21.93998861,
       18.91307123, 27.34919594, 33.28549814, 12.30799153, 35.86658967,
       17.4210999 , 15.02756448, 20.42091072, 18.59287895, 20.7474396 ,
       22.49496374, 19.67629599, 16.27371307, 18.04939953, 21.56688907,
       21.87424989, 18.45322924])

In [9]:
initXML = path+'/spare2_new.xml'
newXML = path+'spare2_optiphi.xml'
pfi = pfiControl.PFI(fpgaHost='localhost', doConnect=False) #'fpga' for real device.
pfi.loadModel(initXML)

model=pfi.calibModel
phiTable = 'phiOntime.csv'

t=Table([pid, model.motorOntimeFwd2, otm.j2fwd_int, otm.j2fwd_slope, newOntimeFwd2,
         model.motorOntimeRev2,otm.j2rev_int, otm.j2rev_slope, newOntimeRev2],
         names=('Fiber No','Ori Fwd OT', 'FWD int', 'FWD slope', 'New Fwd OT',
                'Ori Rev OT', 'REV int', 'REV slope', 'New Rev OT'),
         dtype=('i2','f4', 'f4', 'f4','f4', 'f4', 'f4', 'f4', 'f4'))
t.write(phiTable,format='ascii.ecsv',overwrite=True, 
        formats={'Fiber No':'%i','Ori Fwd OT': '%10.5f', 'FWD int': '%10.5f', 'FWD slope': '%10.5f', 'New Fwd OT': '%10.5f',\
        'Ori Rev OT': '%10.5f', 'REV int': '%10.5f', 'REV slope': '%10.5f', 'New Rev OT': '%10.5f'})

model.updateOntimes(phiFwd=newOntimeFwd2/1000.0, phiRev=newOntimeRev2/1000.0)
model.createCalibrationFile(newXML)

2019-06-04T10:22:24.776 20 fpga       load cobra model from /Users/chyan/Documents/workspace/ics_cobraCharmer/python/ics/cobraCharmer/../../../xml/updatedLinksAndMaps.xml
2019-06-04T10:22:24.903 20 fpga       load cobra model from /Volumes/Disk/Data/20190530/phi//spare2_new.xml


In [63]:
arm='phi'
repeat = 3
    
for tms in tarray:
    for n in range(repeat):
        phi_list = glob.glob(path+f'{tms:0>4d}/{arm}*{n}.fits.gz')
        #print(path+f'{tms:0>4d}/{arm}*{n}.fits.gz')
        a = 1000.0
            
        fig=plt.figure(figsize=(10, 10))
        columns = 1
        rows = len(phi_list)

        ax = []
        i = 0
        for f in phi_list:
            hdu = fits.open(f)
            maxi = 0.5*np.max(hdu[0].data)
            image = np.log10(a*(hdu[0].data/maxi)+1)/np.log10(a)
            #image = hdu[0].data
            basename = os.path.basename(f)

            xsize = [0,2500]
            ysize = [700, 1100]


            ax.append( fig.add_subplot(rows, columns, i+1) )
            if i < len(phi_list)-1:
                ax[-1].get_xaxis().set_ticks([])
            ax[-1].set_title(basename, fontsize = 10)
            plt.imshow(image[ysize[0]:ysize[1],xsize[0]:xsize[1]],cmap='gray')
            i=i+1

        plt.savefig(path+f'{tms:0>4d}/{arm}{tms}Stacked{n}.png')

In [62]:
%matplotlib qt
fitsfile='/Volumes/Disk/Data/20190530/phi/0015/phi1ForwardStack0.fits.gz'
hdu = fits.open(fitsfile)
image = np.log10(a*(hdu[0].data/maxi)+1)/np.log10(a)
xsize = [0,2500]
ysize = [700, 1100]
plt.imshow(image[ysize[0]:ysize[1],xsize[0]:xsize[1]],cmap='gray')
plt.show()